## 航空宇宙情報システム学第二

<h1><center> 第10回 探索による問題解決  </center> </h1>
<h2><center> $N$-パズルを題材に  </center> </h2>


<center>

2023年6月20日

担当教員: 矢入健久

e-mail: yairi@g.ecc.u-tokyo.ac.jp
</center>

# はじめに

今回は、数値計算が中心だった今までの話題からガラッと変わって、アルゴリズム寄りの話をします。

図を使った方が分かりやすいので、スライドを併用します。


# $N$-パズル

3-パズルは、人間にとっては簡単過ぎますが、アルゴリズムを理解するには都合が良いので、$N$-パズルの中で最も優しい3-パズルから説明します。

3-パズルの初期状態は、
$2\times2=4$ のマス目に、それぞれ1, 2, 3 と書かれた3つの四角形のタイルが置かれていて、1つだけ空白のマスがあります。

例えば、
```
2
3 1
```
という初期状態だとします。ここから、空白に隣接する（上下左右に位置する）タイルを空白にスライドさせていって、ゴール状態

```
  1
2 3
```
に達するようにします。

<font color="red">(注)</font> ゴール状態を
```
1 2
3
```
のように、空白を右下にするような流派？もあるようです（むしろ主流？）が、今回はゴール状態は空白が左上にあることとします。

3-パズルの次に簡単な(と言っても、難しいですが) 8-パズルも同様で、
ゴール状態は、
```
  1 2
3 4 5
6 7 8
```
であるとします。


## Python 上での$N$-パズルの状態の表現

$N = d^2 -1 $ とします。つまり、3-パズルの場合は$d=2$、8-パズルの場合は、$d=3$です。

$N$-パズルの各状態は、$d \times d = d^2$ 個のマスのうち、1マスは空白で、他のマスには、1からN-1までの数字が重なりなく配置されています。今回は、これを、Pythonの1次元のリストで表し、<font color="red">空白のマスは数字0を割り当てる</font>こととします。例えば、3-パズルで、
```
2
3 1
```
という状態は、
```
[2, 0, 3, 1]
```
というリストで表します。ただし、後で述べるように、リストは辞書型のキーに使えないという制限があるので、タプルに変換する場合もあります。

**(練習)** リスト `[0,1,2,3]`の要素を並び替えた順列のパターンは、$4!=24$通りありますが、その全てのパターンのリストを作ってください。

**(ヒント)**
* `itertools.permutations`を使えば簡単に求まりますが、練習なので、これを使わずにコードを書いてみてください。
* (効率の問題はありますが、) 関数の再起呼び出しを使うのが多分一番簡単です。例えば、0からn-1 までの数列の順列パターンのリストを返す関数を`permutations(n)`とし、その中で、`permutations(n-1)`を呼び出すようにします。ただし、`pertmutations(1)`は`[[0]]`を返すとします。
* 再起呼び出しを使わずに書くことももちろん可能です。これは、今日のこの後の話題、幅優先探索に深く関連します。

In [ ]:
def permutations(n):
  if n <= 1:
    return [[0]]
  else:
    res = []
    lst = permutations(n-1)
    for l in lst:
      for i in range(len(l)+1):
        l2 = l.copy()
        l2[i:i] = [n-1]
        res.append(l2)
  return res

permutations(4)

[[3, 2, 1, 0],
 [2, 3, 1, 0],
 [2, 1, 3, 0],
 [2, 1, 0, 3],
 [3, 1, 2, 0],
 [1, 3, 2, 0],
 [1, 2, 3, 0],
 [1, 2, 0, 3],
 [3, 1, 0, 2],
 [1, 3, 0, 2],
 [1, 0, 3, 2],
 [1, 0, 2, 3],
 [3, 2, 0, 1],
 [2, 3, 0, 1],
 [2, 0, 3, 1],
 [2, 0, 1, 3],
 [3, 0, 2, 1],
 [0, 3, 2, 1],
 [0, 2, 3, 1],
 [0, 2, 1, 3],
 [3, 0, 1, 2],
 [0, 3, 1, 2],
 [0, 1, 3, 2],
 [0, 1, 2, 3]]

## 解決可能性(ゴール到達可能性)の判定

例えば、
```
  1
3 2
```
という状態からは、どうやっても、ゴール状態
```
  1
2 3
```
に到達することはできません。実際、$4!=24$通りの順列パターンのうち、
ゴール状態に到達できるのは、その半分の$12$通りです。（余談: ちょっと、パラレルワールドみたいじゃないですか？）

与えられた$N$-パズルの状態が、ゴール状態(空白は最左上とする)に到達できるかどうかを判定する方法として、<font color="red">数列の反転回数と空白(0)の最左上マスからのマンハッタン距離の偶奇が一致するかどうか</font>を調べるやり方があります。
詳しくは、スライドの方で説明します。

これを、コードにすると、例えば次のようになります。(3-puzzleだけでなく、8-puzzle, 15-puzzleにも対応します。)



In [ ]:
# N=3,8,15 に対応
def is_solvable(state):
  d = -1
  l = len(state)
  for k in range(2,5):
    if l == k**2:
      d = k
      break
  if d < 2:
    print("Illegal input")
    return
  # 反転回数のカウント
  ct = 0
  for i in range(l-1):
    for j in range(i+1,l):
      if state[i] > state[j]:
        ct += 1
  # 0 のマンハッタン距離
  mhd0 = sum(divmod(state.index(0),d))
  # 判定
  return ((ct+mhd0) % 2 == 0)

## 初期状態（問題）の生成

ランダムに順列を作り、今作った`is_solvable()`で解決可能かどうかを判定することで、$N$-パズルの問題（初期配置）を作ることができます。例えば、こんな感じです。


In [ ]:
import random,math

# ランダムにスタート状態を生成(デフォルトでは3-パズル)
def gen_n_puzzle(N=3):
  if N==8:
    d = 3
  else:
    d = 2
  start =list(range(d**2))
  while True:
    random.shuffle(start)
    if is_solvable(start):
      break
  return start

# 状態を正方形にして表示
def disp_state(seq):
  d = int(math.sqrt(len(seq)))
  for i in range(d):
    for j in range(d):
      print("{:2d}".format(seq[d*i+j]),end="")
    print("")

# 3-パズルの問題を1個生成
print("3-puzzle")
st3 = gen_n_puzzle()
disp_state(st3)

# 8-パズルの問題を1個生成
print("8-puzzle")
st8 = gen_n_puzzle(8)
disp_state(st8)


3-puzzle
 2 1
 0 3
8-puzzle
 3 7 0
 2 8 4
 1 6 5


## 実行可能なアクションを調べる

$N$-パズル問題の場合、「空白マスが上下左右のいずれかに移動するか」あるいは「空白マスが上下左右のいずれのタイルと場所を交換するか」に着目してアクションを定義するのが得策だと考えられます。

そこで、ある状態`state`が与えられたときに、空白が上下左右のいずれに動くことができるかを、"u","d","l","r"のリストで返す関数を作っておきます。

空白のマス目が、$2\times2$または$3\times3$の格子のどこにあるかを知る必要があります。
前述のように、状態`state`は、0から8の数字の順列を表すリストで表現されています。
そこで、リストオブジェクトの`index()`メソッドを使って、リスト内の`0`の位置を求め、さらに、整数同士の割り算の商と剰余を同時に求める`divmod()`関数を使って、`0`(空白マス)の最左上からの2次元座標を求めます。


In [ ]:
def possible_moves(state):
  d = int(math.sqrt(len(state)))
  moves = []
  #print("seq:",seq)
  # 0(空白) の位置を求める
  ix = state.index(0)
  y0,x0 = divmod(ix,d)
  #print("y0:",y0)
  #print("x0:",x0)
  if y0 > 0:
    moves.append("u")
  if y0 < d-1:
    moves.append("d")
  if x0 > 0:
    moves.append("l")
  if x0 < d-1:
    moves.append("r")
  return moves

# 3パズルで試してみる
disp_state(st3)
possible_moves(st3)

 2 1
 0 3


['u', 'r']

**(練習)** 8-パズルの例題についても可能なアクションを調べてください。

In [ ]:
disp_state(st8)
possible_moves(st8)

 3 7 0
 2 8 4
 1 6 5


['d', 'l']

## アクションを適用したときの遷移状態を求める

ある状態に対して、（適用可能な）アクションを適用した結果遷移する次の状態を求める関数を`apply_move(state,move)`として作ります。

正しいプログラミング作法としては、`state`において`move`が適用可能かどうかをチェックするべきですが、コードが長くなるので、上で作った`possible_moves()`に間違いが無いと信じて、チェックを割愛しています。

あと、リストは変更可能(mutable)なので、新しい状態を作るときは、もとの状態を表すリストのコピーを作成してそれに変更を加えるようにします。


In [ ]:
#ある状態に対して、(適用可能な)アクションを適用したときの遷移状態を返す関数
def apply_move(state,m):
  d = int(math.sqrt(len(state)))
  # 現在の状態をコピー(重要)
  newstate = state.copy()
  # "0"の位置を調べる
  ix0 = state.index(0)
  if m == "u":
    dlt = -d
  elif m == "d":
    dlt = d
  elif m == "l":
    dlt = -1
  else:
    dlt = 1
  # "0"と隣の数を入れ替える
  newstate[ix0],newstate[ix0+dlt] = state[ix0+dlt],state[ix0]
  # 新しい状態を返す
  return newstate

# 3パズルで試してみる
disp_state(st3)
moves = possible_moves(st3)
for m in moves:
  print("move:",m)
  next = apply_move(st3,m)
  disp_state(next)

print("")

# 8パズルでも
disp_state(st8)
moves = possible_moves(st8)
for m in moves:
  print("move:",m)
  next = apply_move(st8,m)
  disp_state(next)


 2 1
 0 3
move: u
 0 1
 2 3
move: r
 2 1
 3 0

 3 7 0
 2 8 4
 1 6 5
move: d
 3 7 4
 2 8 0
 1 6 5
move: l
 3 0 7
 2 8 4
 1 6 5


## ある状態から遷移可能な次の状態のリストを生成

上で作った、`possible_moves()`と`apply_move()`を使って、
探索木のあるノード(状態)において、子ノードとなる可能性のある状態のリストを生成する関数を作ります。


In [ ]:
def expand(state):
  lst_next = []
  moves = possible_moves(state)
  for m in moves:
    next = apply_move(state,m)
    lst_next.append(next)
  return lst_next

# テスト
print("3-puzzle")
print("Current state:",st3)
print("Next states:",expand(st3))

print("8-puzzle")
print("Current state:",st8)
print("Next states:",expand(st8))



3-puzzle
Current state: [2, 1, 0, 3]
Next states: [[0, 1, 2, 3], [2, 1, 3, 0]]
8-puzzle
Current state: [3, 7, 0, 2, 8, 4, 1, 6, 5]
Next states: [[3, 7, 4, 2, 8, 0, 1, 6, 5], [3, 0, 7, 2, 8, 4, 1, 6, 5]]


## 探索木の初期化

本当は、クラスを設計してオブジェクト志向で書いた方がエレガントなのですが、今回はなるべく楽をするために、<font color="red">探索木の情報(調査待ちノードのリスト、調査済みノードのリスト、親ノードを格納する辞書オブジェクト)をグローバル変数で持つ</font>ことにします。

また、3-puzzle, 8-puzzle のゴール状態もグローバルに宣言しておきます。

In [ ]:
# 探索木(が持つ情報)を初期化
def init_tree():
  global open,closed,parent
  open,closed,parent = [],[],{}

# 3-puzzle,8-puzzle のゴール状態
goal = {3:list(range(4)), 8:list(range(9))}
# 以下のように宣言しても同じ
# goal = {}
# goal[3] = [0,1,2,3]
# goal[8] = [0,1,2,3,4,5,6,7,8]

# 一応表示
print(goal)


{3: [0, 1, 2, 3], 8: [0, 1, 2, 3, 4, 5, 6, 7, 8]}


# 幅優先・深さ優先探索の実装

探索木では、各ノードが1つの状態に対応します。そして、探索木を使った探索では、2種類の状態(ノード)リストが重要な役割を持ちます。それらを、openリストとclosedリストと呼ぶことにします。

* **openリスト:** 見つかってはいる(存在はわかっている)が、まだ調査していない(訪問していない)ノード・状態を保管しておくためのリスト。

* **closed**リスト: 調査・訪問・展開済みのノード・状態を保管しておくためのリスト。

そして、
* **幅優先探索(BFS)**では、openリストをキュー(FIFO)として使います。
* **深さ優先探索(DFS)**では、openリストをスタック(LIFO)として使います。

言い換えれば、この違いを切り替えることで、幅優先探索と深さ優先探索を一つの関数で書くことができます。

In [ ]:
# 3/8-puzzle を与えられた初期状態から探索する
# - pri = "d" (既定) : 深さ優先
# - pri = "b" : 幅優先
def search(start,pri="d"):
  if len(start) == 9:
    N,d = 8,3
  else:
    N,d = 3,2
  # グローバル変数を初期化
  init_tree()
  # 初期状態をopenリストに追加
  open.append(start)
  # 無限ループ
  while True:
    # もしopenリストが空だったら、すべての状態を探索済み。到達可能な初期状態であれば、起こり得ない。
    if len(open) == 0:
      print("Goal was not found.")
      break
    else:
      # 幅優先: openの先頭から取り出す。深さ優先: openの末尾から取り出す
      if pri == "b": # 幅優先
        node = open.pop(0)
      else: # 深さ優先
        node = open.pop()
      # 現在の状態を表示
      disp_state(node)
      # closed に追加(移動)
      closed.append(node)
      # ゴールだったら
      if node == goal[N]:
        print("Reached goal !")
        break
      # 現ノードを展開
      cand = expand(node)
      # 一つ一つを確認
      for c in cand:
        if c in closed or c in open: # 調査済み、待ち行列に登録済み
          continue
        else:
          open.append(c) # 末尾に付け足し
          # 親を登録。タプルに変換する必要あり。
          parent[tuple(c)] = tuple(node)
      print("open:",len(open))
      print("closed:",len(closed))


また、探索し終わった後に、スタートからゴールまでのパス(解)を求める関数も作っておきます。

In [ ]:
def get_solution():
  if len(closed[0]) == 9:
    N,d = 8,3
  else:
    N,d = 3,2
  state = tuple(goal[N])
  path = []
  while True:
    path = [state] + path
    if not state in parent:
      break
    else:
      state = parent[state]
  return path

## 3-パズルの例題を幅優先・深さ優先で解く

3-パズルを幅優先、深さ優先で解いてみます。
初期状態は、
```
2 0
3 1
```
とします。

In [ ]:
# 幅優先
search([2,0,3,1],pri="b")
# 見つかった解を表示
sol = get_solution()
print("Solution path:")
for s in sol:
  disp_state(s)
  print("")
print("Number of steps:",len(sol))


 2 0
 3 1
open: 2
closed: 1
 2 1
 3 0
open: 2
closed: 2
 0 2
 3 1
open: 2
closed: 3
 2 1
 0 3
open: 2
closed: 4
 3 2
 0 1
open: 2
closed: 5
 0 1
 2 3
Reached goal !
Solution path:
 2 0
 3 1

 2 1
 3 0

 2 1
 0 3

 0 1
 2 3

Number of steps: 4


幅優先探索では、必ず最適解、すなわち、コスト最小のパスが得られます。この場合は4ステップでした。

一方、深さ優先探索では、必ずしも最適解が得られるとは限りません。この例題の場合は、10ステップも掛かる遠回りの解を見つけました。

In [ ]:
# 深さ優先
search([2,0,3,1],pri="d")
# 見つかった解を表示
sol = get_solution()
print("Solution path:")
for s in sol:
  disp_state(s)
  print("")
print("Number of steps:",len(sol))


 2 0
 3 1
open: 2
closed: 1
 0 2
 3 1
open: 2
closed: 2
 3 2
 0 1
open: 2
closed: 3
 3 2
 1 0
open: 2
closed: 4
 3 0
 1 2
open: 2
closed: 5
 0 3
 1 2
open: 2
closed: 6
 1 3
 0 2
open: 2
closed: 7
 1 3
 2 0
open: 2
closed: 8
 1 0
 2 3
open: 2
closed: 9
 0 1
 2 3
Reached goal !
Solution path:
 2 0
 3 1

 0 2
 3 1

 3 2
 0 1

 3 2
 1 0

 3 0
 1 2

 0 3
 1 2

 1 3
 0 2

 1 3
 2 0

 1 0
 2 3

 0 1
 2 3

Number of steps: 10


## 8-パズルを幅優先・深さ優先で解く


**(練習)** 以下の初期状態に対して、幅優先、深さ優先で解いてみてください。


```
3 1 7
6 2 5
4 0 8
```

<font color="red"> **(注意)** Colaboratory で実行すると、深さ優先で約30分、幅優先で約1時間かかります。 </font>

なお、最適解(最小コスト解)は、18ステップになるはずです。


# ヒューリスティック関数

2種類のヒューリスティック関数（ゴールまでのコストの見積もり）を比べてみようと思います。

## $N$-パズルのヒューリスティック関数(1): 不一致タイルの数

まず、ヒューリティクス関数h(x)を「ゴール状態と不一致のタイルの数」とした場合です。

In [ ]:
# ゴール状態と不一致のタイルの数
def num_misplace(state):
  if len(state)==9:
    N = 8
  else:
    N = 3
  ct = 0
  for i in range(len(state)):
    if state[i]==0:
      continue
    if state[i] != goal[N][i]:
      ct += 1
  return ct

In [ ]:
# 3-パズルの例
disp_state(st3)
print(num_misplace(st3))

# 8-パズルの例
disp_state(st8)
print(num_misplace(st8))

 3 2
 0 1
3
 7 4 8
 3 2 5
 1 6 0
6


## $N$-パズルのヒューリスティック関数(2):マンハッタン距離の総和

「各タイルのマンハッタン距離の総和」をヒューリスティック関数とした場合です。

In [ ]:
def sum_mhdist(state):
  if len(state)==9:
    N,d = 8,3
  else:
    N,d = 3,2
  dist = 0
  for a,b in zip(state,goal[N]):
    dy,dx = divmod(abs(a-b),d)
    if a != 0:
      dist += (dx+dy)
  return dist

# 3-パズルの例
disp_state(st3)
print(sum_mhdist(st3))

# 8-パズルの例
disp_state(st8)
print(sum_mhdist(st8))


 2 1
 0 3
1
 3 7 0
 2 8 4
 1 6 5
12


# 貪欲法

割愛します。自分でやってみてください。

# A* アルゴリズム

In [ ]:
# A*アルゴリズムで使う、f,g,h 関数のテーブルを初期化
def init_func_values():
  global fscore,gscore,hscore
  parent,fscore,gscore,hscore = {},{},{},{}


# A*アルゴリズムによる探索
# デフォルトで、ヒューリスティック関数に、不一致タイル数を使用
def search_astar(start,hfun=num_misplace):
  if len(start) == 9:
    N,d = 8,3
  else:
    N,d = 3,2
  # 初期化
  init_tree()
  init_func_values()
  # start状態を登録
  open.append(start)
  gscore[tuple(start)] = 0
  hscore[tuple(start)] = hfun(start)
  fscore[tuple(start)] = gscore[tuple(start)] + hscore[tuple(start)]
  # 無限ループ
  while True:
    # もしopenが空だったら、すべての状態を探索済み
    if len(open) == 0:
      print("Goal was not found.")
      break
    else:
      # open から先頭の状態を取り出し、現状態とする
      node = open.pop(0)
      disp_state(node)
      print("fscore:",fscore[tuple(node)])
      # closed に追加
      closed.append(node)
      # ゴールだったら
      if node == goal[N]:
        print("Reached goal !")
        break
      # 現ノードを展開
      cand = expand(node)
      # 一つ一つを確認
      for c in cand:
        if c in closed or c in open: # 調査済み、待ち行列に登録済み
          continue
        else:
          open.append(c)
          tupc = tuple(c)
          # 親を登録
          parent[tupc] = tuple(node)
          # スコア関数
          gscore[tupc] = gscore[tuple(node)] + 1
          hscore[tupc] = hfun(c)
          fscore[tupc] = gscore[tupc] + hscore[tupc]
          # openリストを並び替え
          open.sort(key = lambda x:fscore[tuple(x)])
      print("open:",len(open))
      print("closed:",len(closed))


## 3-パズルをA* アルゴリズムで解く

幅優先でもすぐに見つかったので、あまり嬉しさはありませんが、少ない訪問数で見つけることができます。

In [ ]:
# 3-puzzle を A-star で解く。ヒューリスティック関数1
search_astar([2,0,3,1])
# 見つかった解を表示
sol = get_solution()
print("Solution path:")
for s in sol:
  disp_state(s)
  print("")
print("Number of steps:",len(sol))

 2 0
 3 1
fscore: 3
open: 2
closed: 1
 2 1
 3 0
fscore: 3
open: 2
closed: 2
 2 1
 0 3
fscore: 3
open: 2
closed: 3
 0 1
 2 3
fscore: 3
Reached goal !
Solution path:
 2 0
 3 1

 2 1
 3 0

 2 1
 0 3

 0 1
 2 3

Number of steps: 4


In [ ]:
# 3-puzzle を A-star で解く。ヒューリスティック関数2
search_astar([2,0,3,1],hfun=sum_mhdist)
# 見つかった解を表示
sol = get_solution()
print("Solution path:")
for s in sol:
  disp_state(s)
  print("")
print("Number of steps:",len(sol))

 2 0
 3 1
fscore: 3
open: 2
closed: 1
 2 1
 3 0
fscore: 3
open: 2
closed: 2
 2 1
 0 3
fscore: 3
open: 2
closed: 3
 0 1
 2 3
fscore: 3
Reached goal !
Solution path:
 2 0
 3 1

 2 1
 3 0

 2 1
 0 3

 0 1
 2 3

Number of steps: 4


# 8-パズルをA* アルゴリズムで解く

いよいよ本領発揮です。特に、1つめの素朴なヒューリスティック関数でも数秒、2つめのヒューリスティック関数を使えば、1秒程度で最適解を見つけることができるはずです。

In [ ]:
# 例題
start8 = [3, 1, 7, 6, 2, 5, 4, 0, 8]

# 8-puzzle を A-star で解く。ヒューリスティック関数1
search_astar(start8)
# 見つかった解を表示
sol = get_solution()
print("Solution path:")
for s in sol:
  disp_state(s)
  print("")
print("Number of steps:",len(sol))

ストリーミング出力は最後の 5000 行に切り捨てられました。
closed: 218
 6 3 7
 0 2 5
 1 4 8
fscore: 14
open: 142
closed: 219
 1 5 2
 3 7 0
 6 4 8
fscore: 14
open: 142
closed: 220
 3 1 7
 5 8 0
 6 4 2
fscore: 14
open: 143
closed: 221
 3 1 7
 8 2 0
 6 4 5
fscore: 14
open: 144
closed: 222
 3 7 2
 6 0 1
 4 8 5
fscore: 14
open: 146
closed: 223
 7 0 5
 3 6 1
 4 2 8
fscore: 14
open: 147
closed: 224
 5 0 1
 3 6 7
 4 2 8
fscore: 14
open: 148
closed: 225
 0 3 7
 6 4 5
 2 1 8
fscore: 14
open: 148
closed: 226
 6 3 7
 2 4 5
 0 1 8
fscore: 14
open: 148
closed: 227
 3 1 7
 5 4 8
 6 2 0
fscore: 14
open: 148
closed: 228
 3 1 7
 8 4 2
 6 5 0
fscore: 14
open: 148
closed: 229
 3 7 2
 6 0 5
 4 1 8
fscore: 14
open: 150
closed: 230
 3 7 2
 6 1 5
 0 4 8
fscore: 14
open: 150
closed: 231
 0 1 7
 3 2 4
 6 8 5
fscore: 14
open: 150
closed: 232
 3 1 7
 0 4 6
 2 8 5
fscore: 14
open: 151
closed: 233
 1 0 2
 3 4 7
 6 8 5
fscore: 14
open: 152
closed: 234
 1 7 2
 3 6 0
 4 8 5
fscore: 14
open: 153
closed: 235
 0 2 1
 3 6 7
 4 8 5
fscore: 14
open: 

In [ ]:
# 8-puzzle を A-star で解く。ヒューリスティック関数2
search_astar(start8,hfun=sum_mhdist)
# 見つかった解を表示
sol = get_solution()
print("Solution path:")
for s in sol:
  disp_state(s)
  print("")
print("Number of steps:",len(sol))

 3 1 7
 6 2 5
 4 0 8
fscore: 9
open: 3
closed: 1
 3 1 7
 6 2 5
 0 4 8
fscore: 9
open: 3
closed: 2
 3 1 7
 0 2 5
 6 4 8
fscore: 9
open: 4
closed: 3
 0 1 7
 3 2 5
 6 4 8
fscore: 9
open: 4
closed: 4
 3 1 7
 2 0 5
 6 4 8
fscore: 9
open: 6
closed: 5
 3 1 7
 2 4 5
 6 0 8
fscore: 9
open: 7
closed: 6
 3 1 7
 6 0 5
 4 2 8
fscore: 11
open: 9
closed: 7
 3 1 7
 6 2 5
 4 8 0
fscore: 11
open: 9
closed: 8
 1 0 7
 3 2 5
 6 4 8
fscore: 11
open: 10
closed: 9
 3 0 7
 2 1 5
 6 4 8
fscore: 11
open: 11
closed: 10
 3 1 7
 2 5 0
 6 4 8
fscore: 11
open: 12
closed: 11
 3 1 7
 2 4 5
 0 6 8
fscore: 11
open: 12
closed: 12
 3 1 7
 2 4 5
 6 8 0
fscore: 11
open: 12
closed: 13
 1 2 7
 3 0 5
 6 4 8
fscore: 11
open: 14
closed: 14
 1 7 0
 3 2 5
 6 4 8
fscore: 11
open: 14
closed: 15
 3 7 0
 2 1 5
 6 4 8
fscore: 11
open: 14
closed: 16
 3 1 0
 2 5 7
 6 4 8
fscore: 11
open: 14
closed: 17
 1 2 7
 3 4 5
 6 0 8
fscore: 11
open: 15
closed: 18
 3 0 7
 6 1 5
 4 2 8
fscore: 13
open: 16
closed: 19
 3 1 7
 0 6 5
 4 2 8
fscore: 13
ope